# Evaluation vertical Prototype

In [19]:
import define
import json
from os import listdir
import csv
import pandas as pd
from elasticsearch import Elasticsearch
es = Elasticsearch(HOST="http://localhost", PORT=9200)
es = Elasticsearch()

In [20]:
# make dictionary with all topics
import xml.etree.ElementTree as ET
tree = ET.parse('topics.xml')
# tree = ET.parse('topics_selected.xml')
topics = tree.findall('topic')

topicsDic = {}

for topic in topics:
    title = topic.find('title').text
    number = topic.find('number').text
    topicsDic[number] = title

topicsDic

{'1': 'Should teachers get tenure?',
 '2': 'Is vaping with e-cigarettes safe?',
 '3': 'Should insider trading be allowed?',
 '4': 'Should corporal punishment be used in schools?',
 '5': 'Should social security be privatized?',
 '6': 'Is a college education worth it?',
 '7': 'Should felons who have completed their sentence be allowed to vote?',
 '8': 'Should abortion be legal?',
 '9': 'Should students have to wear school uniforms?',
 '10': 'Should any vaccines be required for children?',
 '11': 'Should performance-enhancing drugs be accepted in sports?',
 '12': 'Should birth control pills be available over the counter?',
 '13': 'Can alternative energy effectively replace fossil fuels?',
 '14': 'Is sexual orientation determined at birth?',
 '15': 'Should animals be used for scientific or commercial testing?',
 '16': 'Should prescription drugs be advertised directly to consumers?',
 '17': 'Should recreational marijuana be legal?',
 '18': 'Should churches remain tax-exempt?',
 '19': 'Shoul

## Define the search for the vertical prototype

In [21]:
def search_vertical_prototype(query, num_results):
    # make query
    body = {
        "from":0,
        "size":num_results,
        "query": {
            "match": {
                "document_text":query
            }
        }
    }
    res = es.search(index="boromir_index", body=body)
    # get ID's from retrieved documents
    result_ids_scores = []
    results = res.get('hits').get('hits')

    for doc in results:
        id = doc.get('_id')
        score = doc.get('_score')
        result_ids_scores.append([id, score])

    return result_ids_scores

## Make evaluation for query expansion

In [22]:
all_ranks_pro = []
all_ranks_con = []
numbers = []
true_pos_pro = []
true_pos_con = []
avg_rank_pro = []
avg_rank_con = []

result_ids_pro = []
result_ids_con = []

removed_duplicates_pro = 0
removed_duplicates_con = 0

duplicates = []

num_results = 10 # number of result id's we want to get

for topic in topicsDic:
    query = topicsDic.get(topic)
    query_pro = query + " pro"
    query_con = query + " anti"
    dup_result_ids_scores_pro = search_vertical_prototype(query_pro, num_results*100)
    dup_result_ids_scores_con = search_vertical_prototype(query_con, num_results*100)

    result_ids_pro = []
    result_ids_con = []

    # make sure any id is only present in either pro or con:
    for id, score in dup_result_ids_scores_pro:
        include = True
        for _id, _score in dup_result_ids_scores_con:
            if id == _id and score <= _score:
                include = False
                removed_duplicates_con += 1
                break
        if include:
            result_ids_pro.append(id)
        if len(result_ids_pro) == num_results: break
            
    for id, score in dup_result_ids_scores_con:
        include = True
        for _id, _score in dup_result_ids_scores_pro:
            if id == _id and score <= _score:
                include = False
                removed_duplicates_pro += 1
                break
        if include:
            result_ids_con.append(id)
        if len(result_ids_con) == num_results: break
    
    for id in result_ids_pro:
        if id in result_ids_con:
            print(id)
    
    duplicates.append([topic, removed_duplicates_pro, removed_duplicates_con])
    removed_duplicates_pro = 0
    removed_duplicates_con = 0
    no_TPpro = 0
    no_TPcon = 0
    # print(result_ids_pro)
    # print(result_ids_con)
    # print(len(result_ids_pro))
    # print(len(result_ids_con))

        

    print(query, "\n======================================")

    TP_pro = 0
    TP_con = 0
    total_rank_pro = 0 # the smaller the better
    total_rank_con = 0 # the smaller the better

    # check each result id if it fits the topic (pro)
    for id in result_ids_pro:

        path = define.imagePath() + "/images/"  + id[0:3] + "/" + id + "/pages/"
        try:
            file = listdir(path)
            path = path +  file[0] + "/rankings.jsonl"

            lines = []
            for line in open(path, 'r'):
                lines.append(json.loads(line))

            print(lines[0]['query'])

            if lines[0]['topic'] == topic:
                TP_pro += 1
                rank = lines[0]['rank']
                total_rank_pro += rank
        except: print("not found!")

    # check each result id if it fits the topic (con)
    for id in result_ids_con:

        path = define.imagePath() + "/images/"  + id[0:3] + "/" + id + "/pages/"
        try:
            file = listdir(path)
            path = path +  file[0] + "/rankings.jsonl"

            lines = []
            for line in open(path, 'r'):
                lines.append(json.loads(line))

            print(lines[0]['query'])

            if lines[0]['topic'] == topic:
                TP_con += 1
                rank = lines[0]['rank']
                total_rank_con += rank
        except: print("not found2")
    
    # print search performance for this topic
        
    print("Number of pictures that fit to the query (Pro): ", TP_pro, "/", num_results)
    print("Precision (Pro): ", TP_pro/num_results)
    print("Total Rank for Query (Pro): ", total_rank_pro)
    print("....................................")
    print("Number of pictures that fit to the query (Con): ", TP_con, "/", num_results)
    print("Precision (Con): ", TP_con/num_results)
    print("Total Rank for Query (Con): ", total_rank_con, "\n")

    all_ranks_pro.append(total_rank_pro)
    all_ranks_con.append(total_rank_con)
    numbers.append(topic)
    true_pos_pro.append(TP_pro)
    true_pos_con.append(TP_con)
    avg_rank_pro.append(round(total_rank_pro/TP_pro, 2))
    avg_rank_con.append(round(total_rank_con/TP_con, 2))
    
#print(removed_duplicates)
# make final dataframe
df = pd.DataFrame({'Topic': numbers, 'Rank Pro' : all_ranks_pro,  'TP Pro' : true_pos_pro, 'AVG_Rank Pro' : avg_rank_pro, 'Rank Con' : all_ranks_con, 'TP Con' : true_pos_con, 'AVG_Rank Con' : avg_rank_con})
df.to_csv('evaluation_results/0evaluation_topic.tsv', index = False, sep='\t', line_terminator = '\r\n')

C:\Users\thilo\AppData\Local\Temp/ipykernel_8948/672731708.py:12: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.search(index="boromir_index", body=body)


Should teachers get tenure? 
teacher tenure quote
teacher tenure quote
teacher tenure anti
teacher tenure protests
should teachers get tenure?
teacher tenure
teacher tenure good
teacher are to be given a lifelong guarantee of a job
voting machine anti
Privatizing the social security system is bad because the elderly will die
should teachers get tenure?
teacher tenure anti
teacher tenure anti
teacher tenure anti
There are people who have served their sentence and have paid their debt to society and thus earned the right to vote
cell phone radiation anti
cell phone radiation anti
cell phone radiation quote
advertising prescription drugs quote
cell phone radiation quote
Number of pictures that fit to the query (Pro):  8 / 10
Precision (Pro):  0.8
Total Rank for Query (Pro):  558
....................................
Number of pictures that fit to the query (Con):  4 / 10
Precision (Con):  0.4
Total Rank for Query (Con):  118 

Is vaping with e-cigarettes safe? 
e-cigarettes protests
Is vap

### Durchschnittliche Anzahl verworfener Bilder für pro bzw. con

## create HTML

In [21]:
import define

path_to_images = define.imagePath()

In [22]:
htmlEval = '<!DOCTYPE html> <html>     <head>         <meta charset="UTF-8" name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1">         <title>Evaluation</title> <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" integrity="sha384-Gn5384xqQ1aoWXA+058RXPxPg6fy4IWvTNh0E263XmFcJlSAwiGgFAW/dAiS6JXm" crossorigin="anonymous">   <link rel="stylesheet" href="myCss.css">      <script src="https://code.jquery.com/jquery-3.2.1.slim.min.js" integrity="sha384-KJ3o2DKtIkvYIK3UENzmM7KCkRr/rE9/Qpg6aAZGJwFDMVNA/GpGFF93hXpG5KkN" crossorigin="anonymous"></script>   <script src="https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.12.9/umd/popper.min.js" integrity="sha384-ApNbgh9B+Y1QKtv3Rn7W3mgPxhU9K/ScQsAP7hUibX39j7fakFPskvXusvfa0b4Q" crossorigin="anonymous"></script>    <script src="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/js/bootstrap.min.js" integrity="sha384-JZR6Spejh4U02d8jOt6vLEHfe/JQGiRRSQQxSfFWpi1MquVdAyjUar5+76PVCmYl" crossorigin="anonymous"></script> '
htmlEval = htmlEval + '<script language="Javascript" > \n const dataList = [] \n function addDataToList(query, argpro, argcon, pos, neg) {  var { value } = document.getElementById("user_name").elements["user"]   \n  if(value == ""){ alert ("Please fill in User!")} \n  else{ \n dataList.push(query)    \n var { value } = argpro     \n dataList.push(value)    \n var { value } = argcon     \n dataList.push(value)     \n var { value } = pos   \n dataList.push(value)     \n var { value } = neg  \n dataList.push(value)    \n  var { value } = document.getElementById("user_name").elements["user"]   \n  dataList.push(value)       \n dataList.push(" \\n") \n alert("Inhalt von dataList: "+dataList)  } }  \n function downloadFile() { \n const textFile = btoa("query \\t Argumentative images pro \\t Argumentative images con \\t correct positives \\t correct negatives \\t user_name \\n \\t" + dataList.join("\\t")) \n const saveElement = document.createElement("a") \n saveElement.href = `data:text/plain;base64,${textFile}` \n var { value } = document.getElementById("user_name").elements["user"]   \n  saveElement.download = "0evaluation_verticalPrototype_stance_"+value+".tsv" \n document.body.appendChild(saveElement) \n saveElement.click() \n document.body.removeChild(saveElement)  } \n </script>'
htmlEval = htmlEval + '</head> <body>   \n  <div class="form-group mb-5">   \n  <input id="download_btn" type="submit" class="btn btn-primary" onClick="downloadFile(); " value="Exportieren" />    \n  </div>  \n  <div class="container-out"> \n  <form name="username" action="" method="post" id="user_name">   \n  <div class="form-group row">    \n  <label for="username" class="col-sm-2 col-form-label">User: </label>    \n  <div class="col-sm-10"> \n  <select multiple class="form-control" name="user">  \n  <option>Gregor</option> \n  <option>Miriam</option> \n  <option>Shirin</option> \n  <option>Thilo</option>  \n  </select></div> \n  </div>  \n  </form>'
num_results = 10

resultHTML = 'evaluation_results/0evaluation_verticalPrototype.html'

queryListdf = pd.DataFrame(columns = ['topic', 'stance', 'id'])
for tid in topicsDic:
    query = topicsDic.get(tid)
    query_pro = query + " pro"
    query_con = query + " anti"
    dup_result_ids_scores_pro = search_vertical_prototype(query_pro, num_results*100)
    dup_result_ids_scores_con = search_vertical_prototype(query_con, num_results*100)

    result_ids_pro = []
    result_ids_con = []

    # make sure any id is only present in either pro or con:
    for id, score in dup_result_ids_scores_pro:
        include = True
        for _id, _score in dup_result_ids_scores_con:
            if id == _id and score <= _score:
                include = False
                removed_duplicates += 1
                break
        if include:
            result_ids_pro.append(id)
        if len(result_ids_pro) == num_results: break
            
    for id, score in dup_result_ids_scores_con:
        include = True
        for _id, _score in dup_result_ids_scores_pro:
            if id == _id and score <= _score:
                include = False
                removed_duplicates += 1
                break
        if include:
            result_ids_con.append(id)
        if len(result_ids_con) == num_results: break
    
    print(len(dup_result_ids_scores_pro))
    print(len(dup_result_ids_scores_con))
    print(len(result_ids_pro))
    print(len(result_ids_con))

    src_pro = []
    src_con = []
    for ids in result_ids_pro:
        src_pro.append("images/"  + ids[0:3] + "/" + ids + "/image.png")
    for ids in result_ids_con:
        src_con.append("images/"  + ids[0:3] + "/" + ids + "/image.png")

    link_pro = []
    link_con = []
    #open only directory to get link
    for ids in result_ids_pro:
        with open(path_to_images + "/images/" + ids[0:3] + "/" + ids + "/pages/" + listdir(path_to_images + "/images/" + ids[0:3] + "/" + ids + "/pages")[0] + "/page-url.txt") as f:
            this_link = f.read()
            link_pro.append(this_link)

    for ids in result_ids_con:
        with open(path_to_images + "/images/" + ids[0:3] + "/" + ids + "/pages/" + listdir(path_to_images + "/images/" + ids[0:3] + "/" + ids + "/pages")[0] + "/page-url.txt") as f:
            this_link = f.read()
            link_con.append(this_link)

    #data for HTML processing
    # queryList.append(query)
    print(topicsDic.get(tid))
    evaluation = open("evaluation+link.html").read().format(query=query, 
                                    firstpro = src_pro[0],
                                    secondpro = src_pro[1],
                                    thirdpro = src_pro[2],
                                    fourthpro = src_pro[3],
                                    fifthpro = src_pro[4],
                                    sixthpro = src_pro[5], 
                                    seventhpro = src_pro[6], 
                                    eighthpro = src_pro[7], 
                                    ninthpro = src_pro[8], 
                                    tenthpro = src_pro[9], 
                                    firstcon = src_con[0], 
                                    secondcon = src_con[1], 
                                    thirdcon = src_con[2], 
                                    fourthcon = src_con[3], 
                                    fifthcon = src_con[4], 
                                    sixthcon = src_con[5],
                                    seventhcon = src_con[6],
                                    eighthcon = src_con[7], 
                                    ninthcon = src_con[8], 
                                    tenthcon = src_con[9], 
                                    onepro = link_pro[0], 
                                    twopro = link_pro[1], 
                                    threepro = link_pro[2], 
                                    fourpro = link_pro[3], 
                                    fivepro = link_pro[4], 
                                    sixpro = link_pro[5], 
                                    sevenpro = link_pro[6], 
                                    eightpro = link_pro[7], 
                                    ninepro = link_pro[8], 
                                    tenpro = link_pro[9],
                                    onecon = link_con[0],
                                    twocon = link_con[1], 
                                    threecon = link_con[2], 
                                    fourcon = link_con[3], 
                                    fivecon = link_con[4], 
                                    sixcon = link_con[5], 
                                    sevencon = link_con[6], 
                                    eightcon = link_con[7], 
                                    ninecon = link_con[8], 
                                    tencon = link_con[9])

    #print(topic)
    for id in result_ids_pro:
        # print(id)
        queryListdf = queryListdf.append({"topic": query, "stance": "pro", "id": ("/images/"  + id[0:3] + "/" + id + "/pages/")}, ignore_index=True)
    for id in result_ids_con:
        queryListdf = queryListdf.append({"topic": query, "stance": "con", "id": ("/images/"  + id[0:3] + "/" + id + "/pages/")}, ignore_index=True)

    htmlEval = htmlEval + evaluation

    
htmlEval = htmlEval + '</body> </html>'
file = open(resultHTML, 'w')
file.write(htmlEval)
file.close()
    
##debugging/check
#print(queryListdf)
#queryListdf.to_csv("evaluations/0evaluation_verticalPrototype.tsv",index = False,sep='\t',line_terminator='\r\n')

C:\Users\thilo\AppData\Local\Temp/ipykernel_15980/672731708.py:12: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.search(index="boromir_index", body=body)


1000
1000
10
10
Is vaping with e-cigarettes safe?
1000
1000
10
10
Should abortion be legal?
1000
1000
10
10
Should any vaccines be required for children?
1000
1000
10
10
Should churches remain tax-exempt?
1000
1000
10
10
Is a two-state solution an acceptable solution to the Israeli-Palestinian conflict?
1000
1000
10
10
Should people become vegetarian?
1000
1000
10
10
Are social networking sites good for our society?
1000
1000
10
10
Is golf a sport?
1000
1000
10
10
Should bottled water be banned?
1000
1000
10
10
Is homework beneficial?
